# Training Large Deep Learning Recommender Models with NVIDIA HugeCTR and Vertex AI Training

This notebook demonstrates how to use Vertex AI Training to operationalize training and hyperparameter tuning of large scale deep learning models developed with NVIDIA HugeCTR framework.

The notebook compiles prescriptive guidance for the following tasks:

- Building a custom Vertex training container derived from NVIDIA NGC Merlin Training image
- Configuring, submitting and monitoring a Vertex custom training job
- Configuring, submitting and monitoring a Vertex hyperparameter tuning job
- Retrieving and analyzing results of a hyperparameter tuning job

The deep learning model used in this sample is [DeepFM](https://arxiv.org/abs/1703.04247) - a Factorization-Machine based Neural Network for CTR Prediction. The HugeCTR implementation of this model used in this notebook has been configured for the [Criteo dataset](https://ailab.criteo.com/download-criteo-1tb-click-logs-dataset/). 

## Model implementation

[HugeCTR](https://github.com/NVIDIA-Merlin/HugeCTR) is NVIDIA's GPU-accelerated, highly scalable recommender framework. We highly encourage reviewing the [HugeCTR User Guide](https://github.com/NVIDIA-Merlin/HugeCTR/blob/master/docs/hugectr_user_guide.md) before proceeding with this notebook.

NVIDIA HugeCTR facilitates highly scalable implementations of leading deep learning recommender models including Google's [Wide and Deep](https://arxiv.org/abs/1606.07792), Facebook's [DLRM](https://ai.facebook.com/blog/dlrm-an-advanced-open-source-deep-learning-recommendation-model/), and the [DeepFM](https://arxiv.org/abs/1703.04247) model used in this notebook.

A unique feature of HugeCTR is support for model-parallel embedding tables. Applying model-parallelizm for embedding tables enables massive scalability. Industrial grade deep learning recommendation systems most often employ very large embedding tables. User and item embedding tables - cornerstones of any recommender - can easily exceed tens of millions of rows. Without model-parallelizm it would be impossible to fit embedding tables this large in the memory of a single device - especially when using large embeddng vectors. In HugeCTR, embedding tables can span device memory across multiple GPUs on a single node or even across GPUs in a large distributed cluster. 

HugeCTR supports multiple model-parallelizm configurations for embedding tables. Refer to [the HugeCTR API reference](https://github.com/NVIDIA-Merlin/HugeCTR/blob/master/docs/hugectr_layer_book.md#embedding-types-detail) for detailed descriptions. In the DeepFM implementation used in this notebook, we utilize the `LocalizedSlotSparseEmbeddingHash` embedding type. In this embedding type, an embedding table is segmented into multiple slots or feature fields. Each slot stores embeddings for a single categorical feature. A given slot is allocated to a single GPU and it does not span multiple GPUs. However; in a multi GPU environment different slots can be allocated to different GPUs. 

The following diagram demonstrates an example configuration on a single node with multiple GPU - a hardware topology used by Vertex jobs in this notebook.



<img src="./images/deepfm.png" alt="Model parallel embeddings" />


The Criteo dataset has 24 categorical features so there are 24 slots in the embedding table. Cardinalities of categorical variables vary from tens of milions to low teens so the dimensions of slots vary accordingly. Each slot in the embedding table utilizes an embedding vector of the same size. Note that the distribution of slots across GPUs is handled by HugeCTR; you don't have to explicitly pin a slot to a GPU.

Dense layers of the DeepFM models are replicated on all GPUs using a canonical data-parallel pattern. 

You can find the code that implements the DeepFM model in the `src/training/hugectr/trainer/model.py` file. 


## Model training regimen

The training regimen has been optimized for Vertex AI Training. 
- Google Cloud Storage (GCS) and Vertex Training GCS Fuse are used for accessing training and validation data
- A single node, multiple GPUs worker pool is used for Vertex Training jobs.
- Training code has been instrumented to support hyperparameter tuning using Vertex Training Hyperparameter Tuning Job. 

You can find the code that implements the training regimen in the `src/training/hugectr/trainer/task.py` file.

### Training data access

Large deep learning  recommender systems are trained on massive datasets often hundreds of terabytes in size. Maintaining high-throughput when streaming training data to GPU workers is of critical importance. HugeCTR features a highly efficient multi-threaded data reader that parallelizes data reading and model computations. The reader accesses training data through a file system interface. The reader cannot directly access object storage systems like Google Cloud Storage, which is a canonical storage system for large scale training and validation datasets in Vertex AI Training. To expose Google Cloud Storage through a file system interface, the notebook uses an integrated feature of Vertex AI  - Google Cloud Storage FUSE. Vertex AI GCS FUSE provides a high performance file system interface layer to GCS that is self-tuning and requires minimal configuration. The following diagram depicts the training data access configuration:

<img src="./images/gcsfuse.png" alt="GCS Fuse" />


### Vertex AI Training worker pool configuration

HugeCTR supports both single node, multiple GPU configurations and multiple node, multiple GPU distributed cluster topologies. In this sample, we use a single node, multiple GPU configuration. Due to the computational complexity of modern deep learning recommender models we recommend using Vertex Training A2 series machines for large models implemented with HugeCTR. The A2 machines can be configured with up to 16 A100 GPUs, 96 vCPUs, and 1,360GBs RAM. Each A100 GPU has 40GB of device memory. These are powerful configurations that can handle complex models with large embeddings.

In this sample we use the `a2-highgpu-4g` machine type. 

Both custom training and hyperparameter tuning Vertex AI jobs demonstrated in this notebook are configured to use a [custom training container](https://cloud.google.com/vertex-ai/docs/training/containers-overview). The container is a derivative of [NVIDIA NGC Merlin training container](https://ngc.nvidia.com/catalog/containers/nvidia:merlin:merlin-training). 

### HugeCTR hyperparameter tuning with Vertex AI

The training module has been instrumented to support [hyperparameter tuning with Vertex AI](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview). The custom container includes the [cloudml-hypertune package](https://github.com/GoogleCloudPlatform/cloudml-hypertune), which is used to report the results of model evaluations to Vertex AI hypertuning service. The following diagram depicts a training flow implemeted by the training module.


<img src="./images/hugectrtrainer.png" alt="Training regimen" style="height:15%; width:40%"/>


Note that as of HugeCTR v3.2 release, the `hugectr.inference.InferenceSession.evaluate` method used in the trainer module only supports the *AUC* evaluation metric.


## Notebook flow

This notebook assumes that the Criteo dataset has been preprocessed using the preprocessing workflow detailed in the `01-dataset-preprocessing.ipynb` notebook and the resulting Parquet training and validation splits, and processed data schema have been stored in Google Cloud Storage.

As you walk through the notebook you will execute the following steps:
- Configure notebook environment settings like GCP project, compute region, and the GCS locations of training and validation data splits.
- Build a custom Vertex training container based on NVIDIA NGC Merlin Training container
- Configure and submit a Vertex custom training job
- Configure and submit a Vertex hyperparameter training job
- Retrieve the results of the hyperparameter tuning job

In [1]:
import json
import os
import time
import nvtabular as nvt
import shutil

from nvtabular.columns.schema import ColumnSchema, Schema
from nvtabular.tags import Tags
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

## Configure notebook settings
### Set project, region, and Vertex AI settings

In [2]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-central1'

VERTEX_STAGING_BUCKET = 'gs://jk-vertex-merlin'
VERTEX_SA = 'vertex-sa@jk-mlops-dev.iam.gserviceaccount.com'
LOCAL_STAGING_PATH = '/home/jupyter/staging'

### Initialize Vertex AI SDK

In [3]:
aiplatform.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=VERTEX_STAGING_BUCKET
)

### Prepare a local staging area

In [4]:
if os.path.isdir(LOCAL_STAGING_PATH):
    shutil.rmtree(LOCAL_STAGING_PATH)
os.makedirs(LOCAL_STAGING_PATH)

### Set paths to training and validation datasets

In [5]:
DATA_ROOT = 'gs://jk-criteo-bucket/criteo_processed_parquet'
TRAIN_DATA = f'{DATA_ROOT}/train/_file_list.txt'
VALID_DATA = f'{DATA_ROOT}/valid/_file_list.txt'
SCHEMA_PATH = f'{DATA_ROOT}/train/schema.pbtxt'

## Submit a Vertex custom training job

### Prepare a custom training container

In [6]:
IMAGE_NAME = 'hugectr_deepfm'
IMAGE_URI = f'gcr.io/{PROJECT}/{IMAGE_NAME}'
DOCKERFILE = 'src/Dockerfile.hugectr'

In [7]:
#! gcloud builds submit --tag {IMAGE_URI} {DOCKERFILE}

In [8]:
!docker build -t {IMAGE_URI} -f {DOCKERFILE} src
!docker push {IMAGE_URI}

Sending build context to Docker daemon  476.7kB
Step 1/4 : FROM nvcr.io/nvidia/merlin/merlin-training:21.09
 ---> 8f6ef763d770
Step 2/4 : RUN pip3 install cloudml-hypertune
 ---> Using cache
 ---> 7be51676d034
Step 3/4 : WORKDIR /src
 ---> Using cache
 ---> 8e80619f2291
Step 4/4 : COPY training/hugectr/ ./
 ---> Using cache
 ---> 623c6915619a
Successfully built 623c6915619a
Successfully tagged gcr.io/jk-mlops-dev/hugectr_deepfm:latest


### Configure a custom training job

#### Retrieve cardinalities for categorical columns

In [10]:
LOCAL_SCHEMA_PATH = f'{LOCAL_STAGING_PATH}/schema.pbtxt'

!gsutil cp {SCHEMA_PATH} {LOCAL_SCHEMA_PATH}

Copying gs://jk-criteo-bucket/criteo_processed_parquet/train/schema.pbtxt...
/ [1 files][ 20.8 KiB/ 20.8 KiB]                                                
Operation completed over 1 objects/20.8 KiB.                                     


In [11]:
schema = Schema.load_protobuf(LOCAL_SCHEMA_PATH)

In [12]:
def retrieve_cardinalities(schema):
    cardinalities = {key: value.properties['embedding_sizes']['cardinality'] 
                     for key, value in schema.column_schemas.items()
                     if Tags.CATEGORICAL in value.tags}
    
    return cardinalities
    
    
cardinalities = retrieve_cardinalities(schema)
cardinalities

{'C1': 18792578.0,
 'C2': 35176.0,
 'C3': 17091.0,
 'C4': 7383.0,
 'C5': 20154.0,
 'C6': 4.0,
 'C7': 7075.0,
 'C8': 1403.0,
 'C9': 63.0,
 'C10': 12687136.0,
 'C11': 1054830.0,
 'C12': 297377.0,
 'C13': 11.0,
 'C14': 2209.0,
 'C15': 10933.0,
 'C16': 113.0,
 'C17': 4.0,
 'C18': 972.0,
 'C19': 15.0,
 'C20': 19550853.0,
 'C21': 5602712.0,
 'C22': 16779972.0,
 'C23': 375290.0,
 'C24': 12292.0,
 'C25': 101.0,
 'C26': 35.0}

#### Set HugeCTR model and trainer configuration

In [13]:
TRAINING_MODULE = 'trainer.task'

NUM_EPOCHS = 0
MAX_ITERATIONS = 50000
EVAL_INTERVAL = 1000
EVAL_BATCHES = 500
EVAL_BATCHES_FINAL = 2500
DISPLAY_INTERVAL = 200
SNAPSHOT_INTERVAL = 0
WORKSPACE_SIZE_PER_GPU = 61
PER_GPU_BATCHSIZE = 2048
LR = 0.001
DROPOUT_RATE = 0.5
NUM_WORKERS = 12
SLOT_SIZE_ARRAY = json.dumps(
    [int(cardinality) for cardinality in cardinalities.values()]).replace(' ', '')

#### Set training node configuration

In [14]:
MACHINE_TYPE = 'a2-highgpu-4g'
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
ACCELERATOR_NUM = 4

#### Configure worker pool specifications

In [15]:
batchsize = PER_GPU_BATCHSIZE * ACCELERATOR_NUM
gpus = json.dumps([list(range(ACCELERATOR_NUM))]).replace(' ','')
                 
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": MACHINE_TYPE,
            "accelerator_type": ACCELERATOR_TYPE,
            "accelerator_count": ACCELERATOR_NUM,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": ["python", "-m", TRAINING_MODULE],
            "args": [
                '--batchsize=' + str(batchsize),
                '--train_data=' + TRAIN_DATA.replace('gs://', '/gcs/'), 
                '--valid_data=' + VALID_DATA.replace('gs://', '/gcs/'),
                '--slot_size_array=' + SLOT_SIZE_ARRAY,
                '--max_iter=' + str(MAX_ITERATIONS),
                '--max_eval_batches=' + str(EVAL_BATCHES),
                '--eval_batches=' + str(EVAL_BATCHES_FINAL),
                '--dropout_rate=' + str(DROPOUT_RATE),
                '--lr=' + str(LR),
                '--num_workers=' + str(NUM_WORKERS),
                '--num_epochs=' + str(NUM_EPOCHS),
                '--eval_interval=' + str(EVAL_INTERVAL),
                '--snapshot=' + str(SNAPSHOT_INTERVAL),
                '--display_interval=' + str(DISPLAY_INTERVAL),
                '--workspace_size_per_gpu=' + str(WORKSPACE_SIZE_PER_GPU),
                '--gpus=' + gpus,
            ],
        },
    }
]

### Submit and monitor the job

In [16]:
job_name = 'HUGECTR_{}'.format(time.strftime("%Y%m%d_%H%M%S"))
base_output_dir = f'{VERTEX_STAGING_BUCKET}/job_dir/{job_name}'


job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=base_output_dir
)
job.run(
    sync=True,
    service_account=VERTEX_SA,
    restart_job_on_worker_restart=False
)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/6150234838197600256
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-central1/customJobs/6150234838197600256')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6150234838197600256?project=895222332033
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/6150234838197600256 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/6150234838197600256 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/

## Submit and monitor a Vertex hyperparameter tuning job

### Configure a hyperparameter job

#### Set HugeCTR model and trainer configuration

In [ ]:
TRAINING_MODULE = 'trainer.task'

NUM_EPOCHS = 0
MAX_ITERATIONS = 10000
EVAL_INTERVAL = 1000
EVAL_BATCHES = 500
EVAL_BATCHES_FINAL = 2500
DISPLAY_INTERVAL = 200
SNAPSHOT_INTERVAL = 0
WORKSPACE_SIZE_PER_GPU = 61
PER_GPU_BATCHSIZE = 2048
NUM_WORKERS = 12
SLOT_SIZE_ARRAY = json.dumps(
    [int(cardinality) for cardinality in cardinalities.values()]).replace(' ', '')

#### Set training node configuration

In [ ]:
MACHINE_TYPE = 'a2-highgpu-4g'
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
ACCELERATOR_NUM = 4

#### Configure worker pool specification

In [ ]:
batchsize = PER_GPU_BATCHSIZE * ACCELERATOR_NUM
gpus = json.dumps([list(range(ACCELERATOR_NUM))]).replace(' ','')
                 
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": MACHINE_TYPE,
            "accelerator_type": ACCELERATOR_TYPE,
            "accelerator_count": ACCELERATOR_NUM,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": ["python", "-m", TRAINING_MODULE],
            "args": [
                '--batchsize=' + str(batchsize),
                '--train_data=' + TRAIN_DATA.replace('gs://', '/gcs/'), 
                '--valid_data=' + VALID_DATA.replace('gs://', '/gcs/'),
                '--slot_size_array=' + SLOT_SIZE_ARRAY,
                '--max_iter=' + str(MAX_ITERATIONS),
                '--max_eval_batches=' + str(EVAL_BATCHES),
                '--eval_batches=' + str(EVAL_BATCHES_FINAL),
                '--num_workers=' + str(NUM_WORKERS),
                '--num_epochs=' + str(NUM_EPOCHS),
                '--eval_interval=' + str(EVAL_INTERVAL),
                '--snapshot=' + str(SNAPSHOT_INTERVAL),
                '--display_interval=' + str(DISPLAY_INTERVAL),
                '--workspace_size_per_gpu=' + str(WORKSPACE_SIZE_PER_GPU),
                '--gpus=' + gpus,
            ],
        },
    }
]

#### Configure hyperparameter and metric specs

In [ ]:
metric_spec = {'AUC': 'maximize'}

parameter_spec = {
    'lr': hpt.DoubleParameterSpec(min=0.001, max=0.01, scale='log'),
    'dropout_rate': hpt.DiscreteParameterSpec(values=[0.4, 0.5, 0.6], scale=None),
}

### Submit and monitor the job

In [ ]:
job_name = 'HUGECTR_HTUNING_{}'.format(time.strftime("%Y%m%d_%H%M%S"))
base_output_dir = f'{VERTEX_STAGING_BUCKET}/job_dir/{job_name}'


custom_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    base_output_dir=base_output_dir
)

hp_job = aiplatform.HyperparameterTuningJob(
    display_name=job_name,
    custom_job=custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=4,
    parallel_trial_count=2,
    search_algorithm=None)

hp_job.run(
    sync=True,
    service_account=VERTEX_SA,
    restart_job_on_worker_restart=False
)

### Retrieve trial results

In [ ]:
hp_job.trials

#### Find the best trial

In [ ]:
best_trial = sorted(hp_job.trials, 
                    key=lambda trial: trial.final_measurement.metrics[0].value, 
                    reverse=True)[0]

print("Best trial ID:", best_trial.id)
print("   AUC:", best_trial.final_measurement.metrics[0].value)
print("   LR:", best_trial.parameters[1].value)
print("   Dropout rate:", best_trial.parameters[0].value)